<a href="https://colab.research.google.com/github/LuciaMartinezGavier/compromiso-social-estudiantil/blob/main/parafrasis_es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generación de paráfrasis en español


Elegir insecto
Opciones: babosa, oruga, hormigas, babositas

In [1]:
bug = 'oruga'

Obtener los textos que se desean parafrasear

In [2]:
import requests

# url tiene el link a la descarga de un json con todos los párrafos
urls = {
  'babosa':   'https://drive.google.com/uc?export=download&id=1_0DHmdMzrr_0RCbc1-zxMsUNHZ_yTG4X',
  'oruga':    'https://drive.google.com/uc?export=download&id=1UBRHkv7s39ju6fmthJ7lSfQ41Fy6MF8C',
  'hormiga':  'https://drive.google.com/uc?export=download&id=1U63xv_DPTb4FAI9XKiJ9ly6au7FQhNKm',
  'babosita': 'https://drive.google.com/uc?export=download&id=1RzIe2ejobaGxN3m-nrA6p4hyLqdQxjI1'
}

try:
  bug = requests.get(urls[bug]).json()
except:
  print("An error ocurred")


Utilizar modelo para hacer paráfrasis

In [3]:
!pip install transformers==4.10.2
!pip install sentencepiece==0.1.96

In [4]:
# importar modelo de paráfrasis
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert2bert_shared-spanish-finetuned-paus-x-paraphrasing")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/bert2bert_shared-spanish-finetuned-paus-x-paraphrasing")
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print("device ", device)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


device  cpu


In [5]:
from copy import deepcopy
parafrasis = deepcopy(bug)

for topic in bug:
  for paragraph in bug[topic]:
    context = paragraph
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    text = context + " </s>"

    encoding = tokenizer.encode_plus(
        text,
        max_length = 500,
        padding=True,
        return_tensors = "pt"
    )

    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    model.eval()
    beam_outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=500,
        early_stopping=True,
        num_beams=15,
        num_return_sequences=3,
    )

    for beam_output in beam_outputs:
        sent = tokenizer.decode(beam_output, skip_special_tokens = True,
                                clean_up_tokenization_spaces = True)
        parafrasis[topic].append(sent)


In [6]:
for topic in parafrasis:
  for j in parafrasis[topic]:
    print(j)
  print("-------------------------")

Las orugas son larvas de mariposas y polillas. Que se alimentan de hojas, flores, tallos, frutos o raíces. La oruga 'cogollero del maíz' u 'oruga militar  del sur' es las que suele encontrarse en invernaderos, jardines y huertas.
Las orugas son larvas de mariposas y polillas.  Algunas pueden ser plagas importantes para la agricultura, y atacan principalmente al maíz, la soja, el algodón, la alfalfa, el tomate, el lino, la papa y el sorgo.
Las orugas son larvas de mariposas y polillas y representan un problema para las pequeñas huertas y jardines de las casas donde atacan distintas especies de plantas, entre ellas el trébol y los lupinos.
Las orugas son larvas de mariposas y polillas que se alimentan de hojas, flores, tallos, Fructuos, insectos o raíces, la oruga cogollero del sur u las huertas son las que suelen ser nidos.
Las orugas son larvas de mariposas y polillas que se alimentan de hojas, flores, tallos, Fruugas, insectos o raíces, la oruga cogollero del sur u las huertas son las

Generar textos usando los párrafos creados. Un párrafo por tema.

In [8]:
from itertools import product
# combinar paráfrasis
#final_text = [t1 +"\n" + t2 +"\n" + t3 +"\n"+ t4 +"\n" + t5 for t1, t2, t3, t4, t5 in product(parafrasis["que"], parafrasis["como"], parafrasis["habitat"], parafrasis["daño"], parafrasis["prevencion"])]

# combinar párrafos madre
final_text = [
    t1 + "\n" + t2 + "\n" + t3 + "\n"+ t4 + "\n" + t5
    for t1, t2, t3, t4, t5 in product(bug["que"], bug["como"], bug["habitat"],
                                      bug["daño"], bug["prevencion"])
]

import random
for i in range(4):
  num = random.randint(0, len(final_text)-1)
  print(final_text[num] + "\n\n")

Las orugas son larvas de mariposas y polillas. Que se alimentan de hojas, flores, tallos, frutos o raíces. La oruga 'cogollero del maíz' u 'oruga militar  del sur' es las que suele encontrarse en invernaderos, jardines y huertas.
El cuerpo de la oruga se encuentra dividido en una serie de segmentos, los cuales presentan tres pares de patas más varios pares de falsas partas que usan para caminar y agarrarse.
Hay aproximadamente 30 especies de orugas que se distribuyen a lo largo de seis continentes. En América se encuentra en varios países desde Argentina y Chile hasta el sur de Estados Unidos.
La mayoría de las especies de orugas son completamente inofensivas para tocar. Sin embargo, otras orugas que poseen pelos o espinas pueden irritar y dañar la piel. Si usted no está seguro de qué oruga se trata, se aconseja evitar el contacto directo.
El periodo crítico para el control es cuando las plantas aún son muy jóvenes o cuando generan brotes nuevos, ya que las plantas están más vulnerable